# 1.0 A hexagonal analysis grid

A global grid is required to split the analysis of the satellite imagery into smaller areas within which the analysis is conducted. Coast X-Ray makes use of the ISEA3H: Icosahedral Snyder Equal Area Aperture 3 Hexagonal Grid which can be accessed via the R package [dggridR](https://cran.r-project.org/web/packages/dggridR). A hexagonal grid was chosen as this ensures that (almost) all cells are of an equal area, regardless where you are on the globe.

The ISEA3H grid is available at different resolutions, with resolution '12' (cells have an area of approximately 96 km<sup>2</sup>) being the most appropriate for Coast X-Ray analysis. The other resolutions are as follows:

![ISEA3H_res_table.png](./ISEA3H_res_table.png)



## 1.2 Install R Packages

The dggridR package needs to be installed, along with the dplyr, rgdal, ggplot2, and maps. If these are not installed already uncomment the code below and run the code cell.

In [ ]:
## required packages
require("dggridR") 


We can test everything is working is working correctly by creating a grid and mapping the output. Below we create a grid with the resolution of 5, which covers the globe in 2,432 cells. If everything is working, you should see a global map showing green hexagons.

In [ ]:
# Resolution of grid
resolution <- 5

# Contruct the grid
dggs <- dgconstruct(res=resolution)
global <- dgearthgrid(dggs, frame=TRUE)

# Get spatial data for the countries
countries <- map_data("world")

# Create a plot of the grid and world
p<- ggplot() + 
    geom_polygon(data=countries, aes(x=long, y=lat, group=group), fill=NA, color="black")   +
    geom_polygon(data=global,      aes(x=long, y=lat, group=group), fill="green", alpha=0.4) +
    geom_path   (data=global,      aes(x=long, y=lat, group=group), alpha=0.4, color="white")
p

## 1.3 Export a global grid 

If the above map is created successfully, you can export a shapefile grid to a local directory using the code below. This creates a global grid, we will remove the unwanted cells from the global grid for your area of interest in the next step.

In [ ]:
#you may need to increase memory size that R can access when exporting the higher (10 and up) resolution levels
memory.limit(32000)

#set the resolution of the grid (see the ISEA3H information at the top if this doc)
resolution <- 12

#create the global grid
dggs <- dgconstruct(res=resolution)
global <- dgearthgrid(dggs, frame=FALSE)

#set the export path and file name
exportPath <- "./gridsGlobal/ISEA3H_12.shp"

#prepare the grid for output
global.cell <- data.frame(cell=getSpPPolygonsIDSlots(global),row.names=getSpPPolygonsIDSlots(global))
global <- SpatialPolygonsDataFrame(global, global.cell)

for(i in 1:length(global@polygons)) {
  if(max(global@polygons[[i]]@Polygons[[1]]@coords[,1]) - 
     min(global@polygons[[i]]@Polygons[[1]]@coords[,1]) > 270) {
    global@polygons[[i]]@Polygons[[1]]@coords[,1] <- (global@polygons[[i]]@Polygons[[1]]@coords[,1] +360) %% 360
  }
}

#write the shapefile to local
#it takes about three days, to export the 13 grid, an hour for the 12 grid, 10 mins for the 11, and much quicker for the smaller resolutions (computer dependent)
writeOGR(global, exportPath,"", "ESRI Shapefile")

**END**